# Limit LSS covariance matrix check
The GPY test requires to compute the covariance matrix of the LSSs in the limit where N,M tends to infinity at some specified rate M/N=c.

The computation of this matrix is not trivial, so this notebook is used as an additional check. 

In [1]:
from gpy_test import GPY
from gpy_test.result import GPYResult

from statsmodels.tsa.arima_process import arma_generate_sample

from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np

In [45]:
# the complex gaussian time series will be ARMA(1,1) with the following parameters
ar = 0.4
ma = 0.3
N = 1000  # number of samples
M = 5000  # number of time series
burn = 100

c = M / N

# for the GPY test, we will use the following test functions
fs = [lambda x: x, lambda x: x**2]

# Empirical covariance

In [44]:
def run() -> GPYResult:
    real = arma_generate_sample([1, -ar], [1, ma], (N + burn, M), scale=1 / np.sqrt(2))
    imag = arma_generate_sample([1, -ar], [1, ma], (N + burn, M), scale=1 / np.sqrt(2))
    y = real + 1j * imag
    # y = real * np.sqrt(2)
    assert y.shape == (N + burn, M)
    return GPY(y[burn:], fs)


# Generate a large number of experiments
n_repeat = 1000
results = Parallel(n_jobs=-1)(delayed(run)() for _ in tqdm(range(n_repeat)))
LSSs = np.array([result.LSSs_diff for result in results])

100%|██████████| 1000/1000 [00:29<00:00, 34.36it/s]


In [46]:
sample_cov = np.cov(LSSs.T)
sample_cov

array([[   22.68708632,   480.29593824],
       [  480.29593824, 11367.08259404]])

## With Stieltjes transform estimate

In [50]:
real = arma_generate_sample(
    [1, -ar], [1, ma], (N + burn, int(M / 2)), scale=1 / np.sqrt(2)
)
imag = arma_generate_sample(
    [1, -ar], [1, ma], (N + burn, int(M / 2)), scale=1 / np.sqrt(2)
)
y = real + 1j * imag

In [51]:
from gpy_test.covariance import compute_covariance

half_covariance = y.conj().T @ y / N
half_eigenvalues = np.linalg.eigvalsh(half_covariance)

limit_cov = compute_covariance(
    fs,
    XTX_eigenvalues=half_eigenvalues,
    c=int(M / 2) / N,
    covariance_config=None,
)

limit_cov / 2 * 2

array([[   40.15783306,   819.89131786],
       [  819.89131786, 18407.70873714]])

In [52]:
limit_cov / sample_cov

array([[1.7700745 , 1.70705445],
       [1.70705445, 1.61938726]])